In [2]:
import dtlpy as dl
from datetime import datetime

if dl.token_expired():
    dl.login()

In [3]:
project = dl.projects.get(project_id='8c217b4c-53e3-4f05-9205-76bf88ec0838')
dataset = project.datasets.get(dataset_id='65a388a09d96c8a32c7f8b2c')

In [4]:
taggers_company = {'Indivillage': 'raghunath.m@indivillage.com', 'Edgecase': 'nivedreddyk@dyomo.com', 'Apex': 'taranis-apex@aci.apexcovantage.com', 'MTM': 'mauricio@mtmconsult.com.br'}

In [5]:
def filters_funct(crop):
    filters = dl.Filters()
    filters.resource = dl.FiltersResource.ITEM
    filters.add(field='metadata.crop', values=f'{crop}')
    return filters

def filter_grids_annot():
    filters = dl.Filters()
    filters.resource = dl.FiltersResource.ANNOTATION
    filters.add(field='label', values='AI Weed Area Tags.Tagging Grid', operator=dl.FiltersOperations.NOT_EQUAL)
    return filters

def get_image_id(item_name):

    def filter_name(item_name_def):
        filters = dl.Filters()
        filters.resource = dl.FiltersResource.ITEM
        filters.add(field='name', values=item_name_def)
        
        return filters


    filters = filter_name(item_name)
    item_id = [item.metadata['user']['ID'] for page in dataset.items.list(filters=filters) for item in page][0]

    return item_id

In [6]:
def clone_dataset (dataset_task_name, crop):

    filters_crop = filters_funct(crop)
    filters_annot = filter_grids_annot()

    task_dataset = dataset.clone(clone_name=dataset_task_name, filters=filters_crop, with_items_annotations=True, with_metadata=False)
    print ('task_dataset created')

    for page in task_dataset.items.list():
        
        for item in page:

            #Update item_ID
            item_name = item.name
            item_id = get_image_id(item_name)

            item.metadata['user'] = {}
            item.metadata['user']['ID'] = item_id
            item.update()

            #Delete labels (except grids)
            for annot in item.annotations.list(filters=filters_annot):
                annot.delete()
    print ('task_dataset updated')
    return task_dataset

def create_consensus_task (task_dataset, task_name_func, assignee_users_func, task_owner):
    task_dataset.tasks.create(task_name=task_name_func, assignee_ids=assignee_users_func,
                        consensus_percentage=100, consensus_assignees=len(assignee_users_func), task_owner=task_owner)
    print ('Task Created')


In [7]:
project = dl.projects.get(project_id='0c8c900e-468c-4c77-aaab-1871333f772b')
dataset = project.datasets.get(dataset_id='65a2c4f2371e289828389c9f')

In [8]:
taggers = 'Apex'
user_list = ["bhanup@aci.apexcovantage.com","kishoreg@aci.apexcovantage.com","venkateshc@aci.apexcovantage.com"]


crops = ['Corn', 'Soybean']
for crop in crops:

    category = dataset.name.split(' - ')[1]
    task_owner = taggers_company[taggers]
    dataset_task_name = f'{taggers}_{category}_{crop}_{datetime.today().strftime("%Y-%m-%d")}_dt'.lower()
    task_name = dataset_task_name.lower()

    task_dataset = clone_dataset (dataset_task_name, crop)
    create_consensus_task(task_dataset, task_name, user_list, task_owner)

Command Progress: 100%|██████████| 100/100 [06:36<00:00,  3.96s/it]
task_dataset created
Iterate Pages: 100%|██████████| 1/1 [11:42<00:00, 702.45s/it]
task_dataset updated
Command Progress: 100%|██████████| 100/100 [03:12<00:00,  1.92s/it]
Task Created
Command Progress: 100%|██████████| 100/100 [05:15<00:00,  3.15s/it]
task_dataset created
Iterate Pages: 100%|██████████| 1/1 [12:25<00:00, 745.76s/it]
task_dataset updated
Command Progress: 100%|██████████| 100/100 [01:44<00:00,  1.04s/it]
Task Created
